# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [27]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather database/WeatherPy_Database.csv")


# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
0,0,Doume,CM,4.2333,13.4500,68.95,98,83,3.78,moderate rain
1,1,Ambilobe,MG,-13.2000,49.0500,74.59,90,99,2.89,overcast clouds
2,2,Mataura,NZ,-46.1927,168.8643,57.47,98,100,3.89,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,44.26,65,40,26.46,scattered clouds
4,4,Hilo,US,19.7297,-155.0900,83.28,79,32,7.00,scattered clouds
5,5,Shimoda,JP,34.6667,138.9500,52.11,92,100,19.42,light rain
6,6,San Patricio,US,28.0170,-97.5169,94.15,27,0,14.38,clear sky
7,7,Tiblawan,PH,6.4783,126.1083,80.94,79,71,9.40,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,84.36,79,75,9.22,broken clouds
9,9,Ponta Do Sol,PT,32.6667,-17.1000,63.72,77,14,6.06,few clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [28]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [29]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]


# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
1,1,Ambilobe,MG,-13.2000,49.0500,74.59,90,99,2.89,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,83.28,79,32,7.00,scattered clouds
7,7,Tiblawan,PH,6.4783,126.1083,80.94,79,71,9.40,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,84.36,79,75,9.22,broken clouds
11,11,Port Alfred,ZA,-33.5906,26.8910,70.18,91,69,7.00,broken clouds
15,15,Rikitea,PF,-23.1203,-134.9692,78.33,69,89,8.03,overcast clouds
16,16,Touros,BR,-5.1989,-35.4608,79.36,79,8,7.16,clear sky
24,24,Byron Bay,AU,-28.6500,153.6167,71.71,89,81,6.80,broken clouds
25,25,Avarua,CK,-21.2078,-159.7750,84.25,79,20,9.22,light rain
26,26,Souillac,MU,-20.5167,57.5167,76.62,83,45,3.69,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [30]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
1,1,Ambilobe,MG,-13.2000,49.0500,74.59,90,99,2.89,overcast clouds
4,4,Hilo,US,19.7297,-155.0900,83.28,79,32,7.00,scattered clouds
7,7,Tiblawan,PH,6.4783,126.1083,80.94,79,71,9.40,broken clouds
8,8,Vaini,TO,-21.2000,-175.2000,84.36,79,75,9.22,broken clouds
11,11,Port Alfred,ZA,-33.5906,26.8910,70.18,91,69,7.00,broken clouds
15,15,Rikitea,PF,-23.1203,-134.9692,78.33,69,89,8.03,overcast clouds
16,16,Touros,BR,-5.1989,-35.4608,79.36,79,8,7.16,clear sky
24,24,Byron Bay,AU,-28.6500,153.6167,71.71,89,81,6.80,broken clouds
25,25,Avarua,CK,-21.2078,-159.7750,84.25,79,20,9.22,light rain
26,26,Souillac,MU,-20.5167,57.5167,76.62,83,45,3.69,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather", "Lat", "Lng"]].copy()

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [32]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Ambilobe,MG,74.59,overcast clouds,-13.2000,49.0500,
4,Hilo,US,83.28,scattered clouds,19.7297,-155.0900,
7,Tiblawan,PH,80.94,broken clouds,6.4783,126.1083,
8,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
11,Port Alfred,ZA,70.18,broken clouds,-33.5906,26.8910,
15,Rikitea,PF,78.33,overcast clouds,-23.1203,-134.9692,
16,Touros,BR,79.36,clear sky,-5.1989,-35.4608,
24,Byron Bay,AU,71.71,broken clouds,-28.6500,153.6167,
25,Avarua,CK,84.25,light rain,-21.2078,-159.7750,
26,Souillac,MU,76.62,scattered clouds,-20.5167,57.5167,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [33]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [35]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Ambilobe - nearest hotel: Hôtel National
Hilo - nearest hotel: Dolphin Bay Hotel
Tiblawan - nearest hotel: No hotel found
Vaini - nearest hotel: Keleti International Resort
Port Alfred - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Touros - nearest hotel: Pousada Atlântico
Byron Bay - nearest hotel: Lord Byron Resort
Avarua - nearest hotel: Paradise Inn
Souillac - nearest hotel: Hotel SkylineInn
Butaritari - nearest hotel: No hotel found
Aripuana - nearest hotel: No hotel found
Cidreira - nearest hotel: Hotel Castelo
Ambulu - nearest hotel: Hotel Ambulu
Inhambane - nearest hotel: Hotel de Inhambane
Coromandel - nearest hotel: Hotel Rodoviário
Kroya - nearest hotel: Setia Hotel
Payo - nearest hotel: No hotel found
Jatai - nearest hotel: Hotel Rio Claro
Uribia - nearest hotel: No hotel found
Tonantins - nearest hotel: Red Davi
Sur - nearest hotel: Sur Hotel
Kilindoni - nearest hotel: Ibizza Inn
Victoria - nearest hotel: 四季酒店 Four Seaso

Acapulco - nearest hotel: Hotel del Valle
Saint-Joseph - nearest hotel: No hotel found
Los Banos - nearest hotel: Palmridge Private Resort
San Cristobal - nearest hotel: Terra Sur
Sakhon Nakhon - nearest hotel: Imperial Sakon Hotel
Kununurra - nearest hotel: Kununurra Country Club Resort
Caiaponia - nearest hotel: No hotel found
Santa Luzia - nearest hotel: Alcoboca
Jeremie - nearest hotel: Delivrance Hotel
Sembakung - nearest hotel: No hotel found
Chengannur - nearest hotel: Saradhy Hotel
Port-Gentil - nearest hotel: Tara-Mé
Black River - nearest hotel: No hotel found
Marovoay - nearest hotel: No hotel found
Padang - nearest hotel: losmen Surya
Cervantes - nearest hotel: No hotel found
Veraval - nearest hotel: Shyam
Capinopolis - nearest hotel: Hotel Di Roma
Barras - nearest hotel: No hotel found
Buin - nearest hotel: No hotel found
Chimbote - nearest hotel: Hostal Diamante Chimbote
Buala - nearest hotel: No hotel found
Blackwater - nearest hotel: No hotel found
Teshie - nearest hotel

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Ambilobe,MG,74.59,overcast clouds,-13.2000,49.0500,Hôtel National
4,Hilo,US,83.28,scattered clouds,19.7297,-155.0900,Dolphin Bay Hotel
7,Tiblawan,PH,80.94,broken clouds,6.4783,126.1083,No hotel found
8,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti International Resort
11,Port Alfred,ZA,70.18,broken clouds,-33.5906,26.8910,No hotel found
...,...,...,...,...,...,...,...
677,Nisia Floresta,BR,81.41,few clouds,-6.0911,-35.2086,No hotel found
678,Sibolga,ID,75.31,overcast clouds,1.7427,98.7792,Hotel Prima Indah
681,Kotaparh,IN,74.84,broken clouds,19.1500,82.3500,No hotel found
684,Kimbe,PG,81.57,overcast clouds,-5.5502,150.1429,Kimbe Bay Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [36]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"]

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Weather,Lat,Lng,Hotel Name
1,Ambilobe,MG,74.59,overcast clouds,-13.2000,49.0500,Hôtel National
4,Hilo,US,83.28,scattered clouds,19.7297,-155.0900,Dolphin Bay Hotel
8,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti International Resort
15,Rikitea,PF,78.33,overcast clouds,-23.1203,-134.9692,Chez Bianca & Benoit
16,Touros,BR,79.36,clear sky,-5.1989,-35.4608,Pousada Atlântico
...,...,...,...,...,...,...,...
643,Port Macquarie,AU,72.90,moderate rain,-31.4333,152.9167,The Observatory
652,Kokopo,PG,81.28,overcast clouds,-4.3520,152.2633,Hotel Kokopo
675,Mambajao,PH,79.11,broken clouds,9.2504,124.7156,GV Hotel Camigun
678,Sibolga,ID,75.31,overcast clouds,1.7427,98.7792,Hotel Prima Indah


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [37]:
# Create the output File (CSV)
# Set the output file name
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [38]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Country", "Weather"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Weather)

In [41]:
hvplot.save(hotel_map, 'WeatherPy_vacation_map.png')